In [4]:
import numpy as np

In [18]:
class ButterflyOptimization():
    def __init__(self):
        self.objective_function=None
        
    def compile_algorithm(self,obj,dimension,lb,ub,c=0.5,p=0.8,population_size=100,max_iter=100,optimization="minimize"):
        self.objective_function = obj
        self.dimension = dimension
        self.lb=lb
        self.ub=ub
        self.c = c
        self.p = p
        self.max_iter = max_iter
        self.optimization = optimization
        self.population_size=population_size
    
    ## Initialize the populations of butterfly randomly
    def __initialize_population(self):
        return self.lb+np.random.rand(self.population_size,self.dimension)*(self.ub-self.lb)
    
    
    ## Calculate the fragrance using the formula f=c*I^2
    def __calculate_fragrance(self,I,a):
        return self.c*np.power(I,a)
    
    ## This function will find the best butterfly in a specific iteration
    def __find_best_butterfly(self,populations,I):
        if self.optimization == "maximize":
            return np.array(populations[np.argmax(I)]),np.max(I)
        elif self.optimization == "minimize":
            return np.array(populations[np.argmin(I)]),np.min(I)
        else:
            raise Exception(f"Unknown Optimization Criteria: {self.optimization}")
    
    ## This function will update the global best butterfly
    def __update_global_best_butterfly(self,global_best,current_best):
        if global_best == None:
            return current_best
        
        if self.optimization == "maximize":
            return current_best if current_best[1]>=global_best[1] else global_best
        elif self.optimization == "minimize":
            return current_best if current_best[1]<=global_best[1] else global_best
    
    ## This is the population update function:
    ## if r < p:
    ## next_population = current_population + (r^2*best_butterfly-current_population)*fragrance
    ## else:
    ## next_population_i = current_population_i + (r^2*current_population_j-current_population_k)*fragrance
    
    def __update_population(self,current_population,best_butterfly,F):
        next_population = np.empty(current_population.shape)
        
        for i,c_pop in enumerate(current_population):
            r = np.random.rand()
            
            if r < self.p:
                next_population[i,:] = c_pop + (np.square(r) * best_butterfly - c_pop)* F[i]
            else:
                jk = np.random.randint(0,current_population.shape[0],2)
                j = current_population[jk[0]]
                k = current_population[jk[1]]
                
                next_population[i,:] = c_pop + (r**2*j-k) * F[i]
        return np.clip(next_population,self.lb,self.ub)
    
        #r = np.random.rand(self.population_size,1)
        #j = current_population.copy()
        #k = current_population.copy()
        
        #np.random.shuffle(j)
        #np.random.shuffle(k)
        
        #next_population_rlp = current_population + (r**2*best_butterfly-current_population)*np.expand_dims(F,axis=1)
        #next_population_rgp = current_population + (r**2*j-k) * np.expand_dims(F,axis=1)
        #r = r < self.p
        #next_population = r*next_population_rlp+(1-r)*next_population_rgp
        #return np.clip(next_population,lb,ub)
    
    def calculate_optimal_value(self):
        if self.objective_function == None:
            raise Exception("You should compile first.")
        global_best = None
        population = self.__initialize_population()
        a = np.random.rand()
        a = 0.1

        for i in range(self.max_iter):
            ## Calculate the value of stimulus intensity I
            
            I=np.empty([self.population_size])
            for j in range(len(I)):
                I[j]=self.objective_function(population[j])
#             I = np.array([self.objective_function(j) for j in population])
            
            ## Calculate the frangrance
            F = self.__calculate_fragrance(I,a)
            
            ## Get the best index of best butterfly from populations
            best_butterfly=self.__find_best_butterfly(population,I)
            
            global_best = self.__update_global_best_butterfly(global_best,best_butterfly)
            
            ## Update the populations 
            populations = self.__update_population(population,best_butterfly[0],F)
            
            #a = np.random.rand()
            #self.c += 0.025/(self.c*self.max_iter)
        return global_best
    

# Testing the Algorithm with objective function

In [23]:
from benchmarking.benchmark import *
function=[Sphere,Schwefel_1_2,Schwefel_2_22,Step,Rosenbrock,Ackley,SumSquare]

In [25]:
for i in function:
    func=i()
    values=np.empty(30)
    for l in range(30):
        lb = np.array([func.range[0]]).repeat(func.dim)
        ub = np.array([func.range[1]]).repeat(func.dim)
        boa = ButterflyOptimization()
        boa.compile_algorithm(func.get_algorithm(),dimension=func.dim,lb=lb,ub=ub,c=0.01,population_size=10,max_iter=100,optimization="minimize")
        values[l]=boa.calculate_optimal_value()[-1]
    print(f"{func.name}\t ==> Min = {np.min(values)} Max = {np.max(values)} Mean = {np.mean(values)}")

Sphere	 ==> Min = 59303.48631776389 Max = 98590.37470132351 Mean = 77461.5925552047
Schwefel_1_2	 ==> Min = 1476819.042881256 Max = 7637669.93080073 Mean = 3391841.5896080188


C:\Users\mdobn\AppData\Local\Temp\ipykernel_19788\670563861.py:23: RuntimeWarning: invalid value encountered in power
  return self.c*np.power(I,a)


Schwefel_2_22	 ==> Min = -4.769607363507435e+21 Max = -3.377047491273099e+17 Mean = -2.9505668016416647e+20
Step	 ==> Min = 54449.29704405024 Max = 93433.28527035381 Mean = 74914.20454125413
Rosenbrock	 ==> Min = 163047958.4396963 Max = 443066886.05690837 Mean = 304390927.64611024
Ackley	 ==> Min = 20.313527791903606 Max = 21.08514983406862 Mean = 20.810222592938015
SumSquare	 ==> Min = 7378.501634879699 Max = 13944.526225445767 Mean = 11309.362621231643


In [ ]:
Rakibul Islam
Sphere_func ==> Minimum: 6.771317785914812e-47, Maximum: 8.600530807758569e-08, Mean: 2.8668547636970312e-09

step_func ==> Minimum: 1.3635305120989898, Maximum: 7.437202806075672, Mean: 4.602139686476761

schwefel_1_2_func ==> Minimum: 8.647476619816076e-08, Maximum: 95572336.3301623, Mean: 3197542.829429056

ackley_func ==> Minimum: 3197542.829429056, Maximum: 3.522926598454767, Mean: 0.402307611150379

In [ ]:
ackley_func ==> Minimum: 4.440892098500626e-16, Maximum: 0.0001161644353122604, Mean: 4.089796855281473e-06
